# Future Selection Notebook

# Feature Selection

This notebook focuses on selecting the most relevant features influencing hotel booking cancellation.
The dataset used is the output of the feature engineering stage.
Multiple feature selection techniques are applied to ensure robustness and interpretability.
